In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import random

In [5]:
USER_AGENT = "Mozilla/5.0 (Windows NT 5.1; rv:9.0.1) Gecko/20100101 Firefox/9.0.1"
headers = requests.utils.default_headers()
headers.update(
    {
        'User-Agent': USER_AGENT,
    }

)

proxy_list = requests.get('https://raw.githubusercontent.com/clarketm/proxy-list/master/proxy-list.txt')
proxies = re.findall('[0-9]+.[0-9]+.[0-9]+.[0-9]+:[0-9]+.*\n', proxy_list.text)
ip_regex = '[0-9]+.[0-9]+.[0-9]+.[0-9]+:[0-9]+'
df_proxy = pd.DataFrame([{'ip': re.findall(ip_regex, proxy)[0], 'country': proxy.split(' ')[1].split('-')[0], 'anonimity': proxy.split(' ')[1].split('-')[1]} for proxy in proxies])
pool_counter = 0
what_is_my_ip = 'https://api.ipify.org/'

status_proxy = requests.get("https://raw.githubusercontent.com/clarketm/proxy-list/master/proxy-list-status.txt")
success_proxy = [re.sub(':.*', '', proxy) for proxy in status_proxy.text.split('\n') if proxy.endswith('success')]
df_proxy['raw_ip'] = df_proxy.ip.apply(lambda x: re.sub(":.*", '', x))

In [7]:
def getProxies():
    pr_list = []
    counter = 0
    for i, row in df_proxy[df_proxy.raw_ip.isin(success_proxy)].iterrows():
        print(row.ip, counter, df_proxy[df_proxy.raw_ip.isin(success_proxy)].shape[0])
        counter += 1
        proxy = {'https': f'https://{row.ip}'}
        try:
            r = requests.get('https://api.ipify.org/', timeout=2, proxies=proxy, headers=headers)
            if row.ip.startswith(r.text):
                r = requests.get('https://autoplius.lt/skelbimai/volvo-xc60-2-4-l-visureigis-2009-dyzelinas-9742229.html', timeout=2, proxies=proxy, headers=headers)
                pr_list.append(row.ip)
        except:
            continue
    return pr_list

tst_success = getProxies() 
len(tst_success)

147.75.79.37:80 0 149
103.41.212.186:8080 1 149
185.234.217.6:3128 2 149
202.80.231.67:8080 3 149
222.165.214.98:36391 4 149
36.72.236.35:8080 5 149
119.18.152.210:3127 6 149
94.51.83.2:8080 7 149
46.105.226.77:8090 8 149
116.90.229.186:59602 9 149
212.42.113.240:37545 10 149
159.192.243.253:8080 11 149
165.227.86.179:3128 12 149
140.227.176.104:80 13 149
129.205.210.90:65205 14 149
45.163.161.58:8080 15 149
213.6.65.30:8080 16 149
181.10.210.99:44252 17 149
35.204.103.68:3128 18 149
31.209.97.66:57482 19 149
36.66.191.194:33800 20 149
189.198.224.1:80 21 149
36.91.43.218:8080 22 149
91.134.180.0:3128 23 149
178.216.0.168:33364 24 149
200.167.191.101:3128 25 149
94.179.135.230:35313 26 149
93.64.183.162:56508 27 149
203.150.172.151:8080 28 149
117.208.148.72:3128 29 149
86.110.27.165:3128 30 149
41.65.186.112:8080 31 149
176.235.148.38:9090 32 149
79.141.163.68:3128 33 149
195.53.237.122:3128 34 149
38.91.101.30:3128 35 149
203.19.88.51:80 36 149
154.0.15.166:46547 37 149
177.38.243.10

7

In [ ]:
df = pd.DataFrame()
error_log = pd.DataFrame()

sitemap = requests.get('https://autoplius.lt/xml_sitemap/index.xml', headers=headers, timeout=30)
soup = BeautifulSoup(sitemap.text, "html.parser")
ads_sitemap = [tag.text for tag in soup.findAll('loc') if 'ann_list' in tag.text]
for sitemap_url in ads_sitemap:
    print(sitemap_url, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    sitemap_link = requests.get(sitemap_url, headers=headers, timeout=30)
    links_soup = BeautifulSoup(sitemap_link.text, "html.parser")
    products = list(set([(tag.find("xhtml:link", {'hreflang': 'en'})['href'], tag.find('lastmod').text, tag.find('priority').text) for tag in links_soup.findAll('url')]))
    items = []
    for i, url in enumerate(products):
        print(f"{i}/{len(products)} | {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        try:
            proxy = {'https': f'https://{random.choice(tst_success)}'}
            r = requests.get(url[0], proxies=proxy, headers=headers, timeout=2)
        except:
            try:
                r = requests.get(url[0], headers=headers, timeout=2)
            except:
                if len(items):
                    df = df.append(items)
                    df.to_csv('output.csv', index=False)
                    items = []
                continue
        prod_soup = BeautifulSoup(r.text, "html.parser")
        data = {}
        try:
            data['title'] = prod_soup.find(attrs = {'class': 'page-title'}).text.replace('\n', '')
        except:
            data['title'] = ''

        try:
            data['price'] = re.sub('(\n)|([ ]+)', '', prod_soup.find(attrs = {'class': 'price'}).text)
        except:
            data['price'] = 'unknown'

        try:
            data['phone'] = re.sub('(\n)|([ ]+)', '', prod_soup.find(attrs = {'class': 'seller-phone-number'}).text)
        except:
            data['phone'] = ''

        try:
            data['description'] = prod_soup.findAll('div', {'class': 'announcement-description'})[0].text
        except:
            data['description'] = ''

        try:
            data['contact_name'] = prod_soup.find('div', {'class': 'seller-contact-name'}).text
        except:
            data['contact_name'] = ''

        try:
            data['contact_location'] = re.sub('(\n)|([ ]{2,})', '', prod_soup.find('div', {'class': 'seller-contact-location'}).text)
        except:
            data['contact_location'] = ''

        try:
            data['article_id'] = re.sub('(\n)|([ ]+)', '', prod_soup.find('li', {'class': 'announcement-id'}).text)
        except:
            data['article_id'] = ''

        try:
            data['image'] = [re.sub('ann_[0-9]', 'ann_3', re.findall("https://autoplius-img.dgn.lt/.*.jpg", tag['style'])[0]) for tag in prod_soup.findAll('div', {'class': 'thumbnail'})]
        except:
            data['image'] = ''

        try:
            data['breadcrumbs'] = [re.sub('(\n)|([ ]+)', '', tag.text) for tag in prod_soup.findAll('li', {'class': 'crumb'})]
        except:
            data['breadcrumbs'] = ''

        data['sitemap_link'] = sitemap_url
        data['url'] = url[0]
        data['scrape_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        try:
            params_label = [re.sub('(\n)|([ ]{2,})', '', tag.text) for tag in prod_soup.findAll(attrs = {'class': 'parameter-label'})]
            params_value = [re.sub('(\n)|([ ]{2,})', '', tag.text) for tag in prod_soup.findAll(attrs = {'class': 'parameter-value'})]
            params = list(zip(params_label, params_value))
            for p in params:
                data[p[0]] = p[1]
        except Exception as e:
            error_log = error_log.append([{'params': e, 'url': url}])

        try:
            features = []
            for row in prod_soup.findAll('div', {'class': 'feature-row'}):
                feature_label = [re.sub('(\n)|([ ]{2,})', '', tag.text) for tag in row.findAll('div', {'class': 'feature-label'})]
                feature_value = [re.sub('(\n)|([ ]{2,})', '', tag.text) for tag in row.findAll('span', {'class': 'feature-item'})]
                features.append({feature_label[0]: feature_value})

            for row in features:
                data.update(row)
        except Exception as e:
            error_log = error_log.append([{'features': e, 'url': url}])
        try:
            memorized = re.findall('[0-9]+ lankytojai', prod_soup.findAll('div', {'class': 'bookmark-stats-bar'})[0].text)[0]
        except Exception as e:
            memorized = ''
        try:
            update_timestamp = [tag.text for tag in prod_soup.findAll('span', {'class': 'bar-item'}) if 'Atnaujintas' in tag.text][0]
        except Exception as e:
            update_timestamp = ''

        data['memorized'] = memorized
        data['update_timestamp'] = update_timestamp

        data['last_updated'] = url[1]
        data['adv_priority'] = url[2]

        items.append(data)

    df = df.append(items)
    df.to_csv('output.csv', index=False)

https://autoplius.lt/xml_sitemap/ann_list/1.xml 2020-01-24 14:19:04
0/5000 | 2020-01-24 14:19:21
1/5000 | 2020-01-24 14:19:24
2/5000 | 2020-01-24 14:19:27
3/5000 | 2020-01-24 14:19:29
4/5000 | 2020-01-24 14:19:31
5/5000 | 2020-01-24 14:19:34
6/5000 | 2020-01-24 14:19:34
7/5000 | 2020-01-24 14:19:37
8/5000 | 2020-01-24 14:19:42
9/5000 | 2020-01-24 14:19:43
10/5000 | 2020-01-24 14:19:44
11/5000 | 2020-01-24 14:19:45
12/5000 | 2020-01-24 14:19:47
13/5000 | 2020-01-24 14:19:49
14/5000 | 2020-01-24 14:19:49
15/5000 | 2020-01-24 14:19:52
16/5000 | 2020-01-24 14:19:53
17/5000 | 2020-01-24 14:19:55
18/5000 | 2020-01-24 14:19:56
19/5000 | 2020-01-24 14:19:57
20/5000 | 2020-01-24 14:19:59
21/5000 | 2020-01-24 14:20:01
22/5000 | 2020-01-24 14:20:03
23/5000 | 2020-01-24 14:20:07
24/5000 | 2020-01-24 14:20:08
25/5000 | 2020-01-24 14:20:10
26/5000 | 2020-01-24 14:20:14
27/5000 | 2020-01-24 14:20:17
28/5000 | 2020-01-24 14:20:18
29/5000 | 2020-01-24 14:20:22
30/5000 | 2020-01-24 14:20:24
31/5000 | 20

/home/domantas/.local/lib/python3.6/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


249/5000 | 2020-01-24 17:53:39
250/5000 | 2020-01-24 17:53:44
251/5000 | 2020-01-24 17:53:46
252/5000 | 2020-01-24 17:53:52
253/5000 | 2020-01-24 17:54:00
254/5000 | 2020-01-24 17:54:04
255/5000 | 2020-01-24 17:54:10
256/5000 | 2020-01-24 17:54:13
257/5000 | 2020-01-24 17:54:16
258/5000 | 2020-01-24 17:54:25
259/5000 | 2020-01-24 17:54:31
260/5000 | 2020-01-24 17:54:39
261/5000 | 2020-01-24 17:54:45
262/5000 | 2020-01-24 17:54:47
263/5000 | 2020-01-24 17:54:51
264/5000 | 2020-01-24 17:54:54
265/5000 | 2020-01-24 17:54:57
266/5000 | 2020-01-24 17:54:59
267/5000 | 2020-01-24 17:55:06
268/5000 | 2020-01-24 17:55:07
269/5000 | 2020-01-24 17:55:10
270/5000 | 2020-01-24 17:55:15
271/5000 | 2020-01-24 17:55:17
272/5000 | 2020-01-24 17:55:22
273/5000 | 2020-01-24 17:55:25
274/5000 | 2020-01-24 17:55:25
275/5000 | 2020-01-24 17:55:28
276/5000 | 2020-01-24 17:55:29
277/5000 | 2020-01-24 17:55:33
278/5000 | 2020-01-24 17:55:38
279/5000 | 2020-01-24 17:55:39
280/5000 | 2020-01-24 17:55:42
281/5000

In [40]:
r = requests.get('https://en.autoplius.lt/ads/mak-r15-5x114-3mm-et40-j7-light-alloy-r15-726698.html', headers=headers)
r.status_code

0       nagotiate
1       nagotiate
2       nagotiate
3       nagotiate
4       nagotiate
          ...    
4995    nagotiate
4996    nagotiate
4997    nagotiate
4998    nagotiate
4999    nagotiate
Name: price, Length: 5000, dtype: object